In [1]:
import nltk
import numpy as np
import random
import string
import bs4 as bs
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import requests
from textblob import TextBlob
from fuzzywuzzy import fuzz
from urllib.request import urlopen

In [2]:
print("Hi, welcome to Cooking Bot! Enter your recipe link!")
link=input()
#https://www.allrecipes.com/chicken-ricotta-meatballs-recipe-8683251

Hi, welcome to Cooking Bot! Enter your recipe link!


 https://www.allrecipes.com/chicken-ricotta-meatballs-recipe-8683251


In [3]:
get_link = urllib.request.urlopen(link)
get_link = get_link.read()

data = bs.BeautifulSoup(get_link, 'lxml')

data_paragraphs = data.find_all('p')

data_text = ''
for para in data_paragraphs:
    data_text +=  para.text 

data_text = data_text.lower()

In [4]:
json_data = {
    'title': data.h1.text,
    'description': data.p.text,
    'items': [li.text for li in data.find_all('li')]
}


In [5]:
html = urlopen('https://www.allrecipes.com/chicken-ricotta-meatballs-recipe-8683251')
bsh = BeautifulSoup(html.read(), 'html.parser')
head=str(bsh.h1)
match = re.search(r'>([^<]+)<', head)
extracted_text= match.group(1).strip()
extracted_text

'Chicken Ricotta Meatballs'

In [6]:
print("Bot: Alright. So let's start working with "+ extracted_text+  ". Lets proceed!")


Bot: Alright. So let's start working with Chicken Ricotta Meatballs. Lets proceed!


In [7]:
# Extract the 'items' list
items_list = json_data['items']

In [8]:
items_list[1:10]

['\n \n\n\n\nLog In\n\n ',
 '\n\n\n\n\nMy Account\n\n\n\n\n\n\n\n Log Out\n \n\n My Profile\n \n\n Saved Recipes & Collections\n \n\n Add a Recipe\n \n\n Help\n \n\n\n',
 '\n Log Out\n ',
 '\n My Profile\n ',
 '\n Saved Recipes & Collections\n ',
 '\n Add a Recipe\n ',
 '\n Help\n ',
 '\n\nMagazine\n\n\n\n\n\n\n\n Subscribe\n \n\n Manage Your Subscription\n \n\n Give a Gift Subscription\n \n\n Get Help\n \n\n\n',
 '\n Subscribe\n ']

In [9]:
def clean_and_filter_strings(string_list):
    cleaned_list = []
    for text in string_list:
        # Remove newline characters and extra spaces
        cleaned_text = ' '.join(text.split())
        # Filter out single-word strings
        if len(cleaned_text.split()) > 0:
            cleaned_list.append(cleaned_text.lower())
    return cleaned_list

In [10]:
# Clean and filter the strings
items_list = clean_and_filter_strings(items_list)

In [11]:
def recipe_extraction(string_list):
    result = []
    
    # Iterate from the end of the list
    for text in reversed(string_list):
        
        
        
        # Check if the length is more than 40
        if len(text) > 40:
            result.append(text)
            
        elif result:
            # Stop appending once we hit a string of length <= 40
            break
    
    # Reverse the final result list
    result.reverse()
    return result

In [12]:
items_list = recipe_extraction(items_list)

In [13]:
def add_step_numbers(string_list):
    # Create a new list with "step N: " added to each string
    stepped_list = [f'cooking step {i+1}: {text}' for i, text in enumerate(string_list)]
    return stepped_list

In [14]:
# Add step numbers
items_list = add_step_numbers(items_list)

In [15]:
items_list

['cooking step 1: preheat the oven to 350 degrees f (175 degrees c). line a baking sheet with foil or parchment paper.',
 'cooking step 2: heat olive oil in a skillet. add onion and garlic, season with 1/4 teaspoon salt and 1/4 teaspoon pepper, and cook until soft and translucent, 3 to 5 minutes. let cool to room temperature.',
 'cooking step 3: combine panko and ricotta in a bowl and mix well. add egg and season with remaining salt, pepper and italian seasoning; mix to combine. add ground chicken, and cooked onion and garlic mixture; mix gently to combine.',
 'cooking step 4: shape into about 24 meatballs and place on the prepared baking sheet.',
 'cooking step 5: bake meatballs in the preheated oven until no longer pink in the center and the juices run clear. an instant-read thermometer inserted into the center should read at least 165 degrees f (74 degrees c), about 20 minutes; remove to cool.',
 'cooking step 6: to make the sauce, preheat the oven to 425 degrees f (220 degrees c).'

In [16]:
data_text[1:500]

'hese ricotta chicken meatballs are both budget friendly and a lighter alternative to the traditional beef meatball, while still being flavorful and satisfying. the roasted tomato sauce is a delicious accompaniment.meatballs2 teaspoons oil1/2 white onion, finely diced6 cloves garlic, minced1/2 teaspoon freshly ground black pepper, divided3/4 teaspoon salt, divided1/2 cup panko1/4 cup ricotta cheese1 large egg1/2 teaspoon italian seasoning1 pound ground chickensauce15 roma tomatoes, cut in half l'

In [17]:
text_list=[]
for para in data_paragraphs:
    text_list.append(para.text)

In [18]:
#text_list

In [19]:
ingredients='recipe ingredients are: '
ingredients_list=[]
for i in text_list:
    if len(i)<40:
        ingredients=ingredients+' '+i
        ingredients_list.append(i.lower())
        
ingredients=ingredients+'.'

In [20]:
ingredients

'recipe ingredients are:  Meatballs 2 teaspoons oil 1/2 white onion, finely diced 6 cloves garlic, minced 3/4 teaspoon salt, divided 1/2 cup panko 1/4 cup ricotta cheese 1 large egg 1/2 teaspoon Italian seasoning 1 pound ground chicken Sauce 1 onion, quartered 2 tablespoons olive oil 8 cloves garlic, peeled 1/2 teaspoon salt, or to taste shaved Parmesan, for garnish TheDailyGourmet.'

In [21]:
#data_text= data_text+' . '+ingredients_list

In [22]:
ingredients_list

['meatballs',
 '2 teaspoons oil',
 '1/2 white onion, finely diced',
 '6 cloves garlic, minced',
 '3/4 teaspoon salt, divided',
 '1/2 cup panko',
 '1/4 cup ricotta cheese',
 '1 large egg',
 '1/2 teaspoon italian seasoning',
 '1 pound ground chicken',
 'sauce',
 '1 onion, quartered',
 '2 tablespoons olive oil',
 '8 cloves garlic, peeled',
 '1/2 teaspoon salt, or to taste',
 'shaved parmesan, for garnish',
 'thedailygourmet']

In [23]:
# Initialize the new list
steps = []
filtered_list = [text for text in text_list if text.endswith('\n')]


In [24]:
filtered_list = [text for text in filtered_list if len(text) > 50]

In [25]:
# Iterate over the original list with enumeration to add step numbers
for i, text in enumerate(filtered_list, start=1):
    step_text = f"Recipe Step {i}: {text.strip()}"
    steps.append(step_text.lower())

In [26]:
# Tokenize sentences
sen = nltk.sent_tokenize(data_text)

# Tokenize words
words = nltk.word_tokenize(data_text)

In [27]:
sen=sen+items_list+ingredients_list

In [28]:
items_list_text=''.join(items_list)

In [29]:
def parse_steps(text):
    step_pattern = re.compile(r'(step\s*\d+:)(.*?)(?=step\s*\d+:|$)', re.IGNORECASE | re.DOTALL)
    steps = []
    for match in step_pattern.finditer(text):
        step_desc = match.group(2).strip()
        steps.append(step_desc)
    return steps

In [30]:
steps = parse_steps(items_list_text)

In [31]:
class RecipeNavigator:
    def __init__(self, steps):
        self.steps = steps
        self.current_step = 0

    def next_step(self):
        if self.current_step < len(self.steps) - 1:
            self.current_step += 1
        self.show_current_step()

    def previous_step(self):
        if self.current_step > 0:
            self.current_step -= 1
        self.show_current_step()

    def show_current_step(self):
        step = self.steps[self.current_step]
        print(f"Step {self.current_step + 1}: {step}")

    def go_to_step(self, step_number):
        if 0 <= step_number < len(self.steps):
            self.current_step = step_number
        self.show_current_step()

In [32]:
navigator = RecipeNavigator(steps)

In [33]:
#navigator.show_current_step()

In [34]:
#navigator.next_step()

In [35]:
#navigator.previous_step()

In [36]:
#navigator.go_to_step(2)

In [37]:
wnlem = nltk.stem.WordNetLemmatizer()

In [38]:
def perform_lemmatization(tokens):
    return [wnlem.lemmatize(token) for token in tokens]


In [39]:
pr = dict ((ord(punctuation), None) for punctuation in string.punctuation) 

In [40]:
#tokenizer

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(pr)))



In [41]:

greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi cooking bot", "whats up")
greeting_responses = ["hey", "hey are you?", "", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [42]:
def generate_response(user_input):
    bot_response = ''
    sen.append(user_input)
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    word_vectors = word_vectorizer.fit_transform(sen)
    similar_vector_values=cosine_similarity(word_vectors[-1],word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]
    
    if vector_matched == 0:
        bot_response = bot_response + "I am sorry I don't understand, check here: "
        query = user_input.replace(" ", "+"); url = f"https://www.google.com/search?q={query}"; headers = {"User-Agent": "Mozilla/5.0"}; response = requests.get(url, headers=headers); soup = BeautifulSoup(response.text, 'html.parser'); links = [a['href'].split('/url?q=')[1].split('&')[0] for a in soup.find_all('a', href=True) if a['href'].startswith('/url?q=')][:3];
        bot_response=bot_response+links[2]

        return bot_response
    else:
        bot_response = bot_response + sen[similar_sentence_number]
        return bot_response


In [43]:
import warnings
warnings.filterwarnings('ignore')

continue_flag = True
print("Hello I am Cooking Bot. Shoot your question")

while(continue_flag == True):
    human = input()
    human = human.lower()
    
    if human != 'bye':
        if human == 'thanks' or human == 'thank you':
            continue_flag = False
            print("Most welcome from Cooking Bot")
        
        elif (human == 'give cooking steps?' or (fuzz.ratio(human, 'give steps to cook the recipe?')>95) or (fuzz.ratio(human, 'how to cook?')>70) or (fuzz.ratio(human, 'what are the cooking steps?')>95)):
            continue_flag = True
            print("Bot: ")
            for i in items_list:
                print (i)

        elif (human == 'Take me to the current step' or (fuzz.ratio(human, 'Take me to the current step?')>95) or (fuzz.ratio(human, 'what is the current step?')>70) ):
            continue_flag = True
            print("Bot: ")
            navigator.show_current_step()

        elif (human == 'Take me to the next step' or (fuzz.ratio(human, 'Take me to the next step?')>95) or (fuzz.ratio(human, 'what is the next step?')>70) ):
            continue_flag = True
            print("Bot: ")
            navigator.next_step()

        elif (human == 'Take me to the previous step' or (fuzz.ratio(human, 'Take me to the previous step?')>95) or (fuzz.ratio(human, 'what is the previous step?')>70) ):
            continue_flag = True
            print("Bot: ")
            navigator.previous_step()

        elif ((fuzz.ratio(human, 'Take me to step 1')>90) or (fuzz.ratio(human, 'what is step 1')>90) ):
            continue_flag = True
            print("Bot: ")
            n=int(human[-1])
            navigator.go_to_step(n-1)
        
        elif (fuzz.ratio(human, 'give all recipe information from website')>70) or (fuzz.ratio(human, 'give recipe content from website')>70):
            continue_flag = True
            print("Bot: ")
            for step in steps:
                print(step)
            
        elif (fuzz.ratio(human, 'give all recipe ingredients')>70) or (fuzz.ratio(human, 'what are the recipe ingredients')>70) or (fuzz.ratio(human, 'what is used to make this dish')>70):
            continue_flag = True
            print("Bot: ")
            print("The ingredients are:")
            for a in ingredients_list:
                print (a)

        
        else:
            if generate_greeting_response(human) != None:
                print("Bot: " + generate_greeting_response(human))
            else:
                print("Bot: ", end="")
                print(generate_response(human))
                sen.remove(human)
    else:
        continue_flag = False
        print("Cooking Bot says Goodbye")

Hello I am Cooking Bot. Shoot your question


 hi


Bot: I am sorry I don't understand, check here: https://www.merriam-webster.com/dictionary/hi


 give cooking steps?


Bot: 
cooking step 1: preheat the oven to 350 degrees f (175 degrees c). line a baking sheet with foil or parchment paper.
cooking step 2: heat olive oil in a skillet. add onion and garlic, season with 1/4 teaspoon salt and 1/4 teaspoon pepper, and cook until soft and translucent, 3 to 5 minutes. let cool to room temperature.
cooking step 3: combine panko and ricotta in a bowl and mix well. add egg and season with remaining salt, pepper and italian seasoning; mix to combine. add ground chicken, and cooked onion and garlic mixture; mix gently to combine.
cooking step 4: shape into about 24 meatballs and place on the prepared baking sheet.
cooking step 5: bake meatballs in the preheated oven until no longer pink in the center and the juices run clear. an instant-read thermometer inserted into the center should read at least 165 degrees f (74 degrees c), about 20 minutes; remove to cool.
cooking step 6: to make the sauce, preheat the oven to 425 degrees f (220 degrees c).
cooking step 7: 

...

Bot: heat olive oil in a skillet.


 what should be pan heat?


Bot: heat olive oil in a skillet.


 bye


Cooking Bot says Goodbye
